In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [ ]:
# Define the list of Indian stocks with correct NSE suffixes
stocks = [
    'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS', 'HINDUNILVR.NS', 'HDFC.NS', 'KOTAKBANK.NS', 'BHARTIARTL.NS', 'LT.NS',
    'ITC.NS', 'ASIANPAINT.NS', 'SBIN.NS', 'BAJFINANCE.NS', 'HCLTECH.NS', 'AXISBANK.NS', 'DMART.NS', 'MARUTI.NS', 'SUNPHARMA.NS', 'ADANIGREEN.NS',
    'ONGC.NS', 'NTPC.NS', 'POWERGRID.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'ULTRACEMCO.NS', 'GRASIM.NS', 'BAJAJFINSV.NS', 'BRITANNIA.NS', 'ADANIPORTS.NS',
    'HEROMOTOCO.NS', 'WIPRO.NS', 'DRREDDY.NS', 'HDFCLIFE.NS', 'TECHM.NS', 'SBICARD.NS', 'SHREECEM.NS', 'CIPLA.NS', 'NESTLEIND.NS', 'DABUR.NS',
    'VEDL.NS', 'INDUSINDBK.NS', 'HINDZINC.NS', 'BOSCHLTD.NS', 'COALINDIA.NS', 'PIDILITIND.NS', 'DIVISLAB.NS', 'ADANIENT.NS', 'BAJAJHLDNG.NS', 'LUPIN.NS',
    'ICICIPRULI.NS', 'APOLLOHOSP.NS', 'COLPAL.NS', 'GODREJCP.NS', 'IOC.NS', 'M&M.NS', 'NAUKRI.NS', 'HAVELLS.NS', 'PGHH.NS', 'EICHERMOT.NS',
    'BPCL.NS', 'SIEMENS.NS', 'BIOCON.NS', 'TATAPOWER.NS', 'BERGEPAINT.NS', 'SRF.NS', 'CONCOR.NS', 'AMBUJACEM.NS', 'GLAND.NS', 'INDIGO.NS',
    'MANAPPURAM.NS', 'MCDOWELL-N.NS', 'LICI.NS', 'HDFCAMC.NS', 'DLF.NS', 'BANDHANBNK.NS', 'AUROPHARMA.NS', 'MPHASIS.NS',
    'TORNTPHARM.NS', 'TRENT.NS', 'PETRONET.NS', 'CUB.NS', 'GMRINFRA.NS', 'BAJAJ-AUTO.NS', 'ABBOTINDIA.NS', 'NMDC.NS', 'TITAN.NS', 'BEL.NS',
    'HINDALCO.NS', 'VOLTAS.NS', 'MUTHOOTFIN.NS', 'BEL.NS', 'EXIDEIND.NS', 'NATIONALUM.NS', 'BANKBARODA.NS', 'SAIL.NS', 'HINDCOPPER.NS', 'SUNTV.NS'
]

In [ ]:
# Define the date range
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)

In [ ]:
# Fetch historical data
def get_stock_data(stocks, start_date, end_date):
    stock_data = {}
    for stock in stocks:
        data = yf.download(stock, start=start_date, end=end_date)
        if not data.empty:
            stock_data[stock] = data
            print(f"Fetched data for {stock}:\n", data.head())
    return stock_data

In [ ]:
def calc_RSI(data, window=14):
    """
    Calculate the Relative Strength Index (RSI) for a given DataFrame.
    
    Parameters:
    - data: pandas DataFrame with a 'Close' column.
    - window: integer, the period over which RSI is calculated.
    
    Returns:
    - rsi: pandas Series containing RSI values.
    """
    # Calculate the differences in the 'Close' column
    delta = data['Close'].diff()
    
    # Calculate gains (only positive changes) and losses (only negative changes)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # Calculate the rolling average of gains and losses
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    
    # Calculate the Relative Strength (RS)
    rs = avg_gain / avg_loss
    
    # Calculate the RSI using the RS
    rsi = 100 - (100 / (1 + rs))
    
    # Replace NaN values in RSI with 50 (a neutral value) if desired, or with forward fill
    rsi = rsi.fillna(50)
    
    print("RSI:\n", rsi.head())
    
    return rsi


In [ ]:
# calc macd
def calc_MACD(data, short_window=12, long_window=26, signal_window=9):
    """
    Calculate the MACD (Moving Average Convergence Divergence) and its Signal Line for a given DataFrame.
    
    Parameters:
    - data: pandas DataFrame with a 'Close' column.
    - short_window: integer, the short window period for the EMA calculation.
    - long_window: integer, the long window period for the EMA calculation.
    - signal_window: integer, the window period for the Signal Line calculation.
    
    Returns:
    - macd: pandas Series containing MACD values.
    - signal: pandas Series containing Signal Line values.
    """
    # Calculate the Short EMA
    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
    
    # Calculate the Long EMA
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
    
    # Calculate the MACD line
    macd = short_ema - long_ema
    
    # Calculate the Signal Line
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    
    # Print MACD and Signal Line for visualization
    print("MACD:\n", macd.head())
    print("Signal Line:\n", signal.head())
    
    return macd, signal

# Example usage:
# df = pd.DataFrame({'Close': [your_data_here]})
# macd, signal = calc_MACD(df)


In [ ]:
# Calculate Bollinger Bands
def bb(data, window=20):
    """
    Calculate the Bollinger Bands for a given DataFrame.
    
    Parameters:
    - data: pandas DataFrame with a 'Close' column.
    - window: integer, the window period for the rolling mean and standard deviation.
    
    Returns:
    - sma: pandas Series containing the Simple Moving Average (SMA).
    - upper_band: pandas Series containing the Upper Bollinger Band values.
    - lower_band: pandas Series containing the Lower Bollinger Band values.
    """
    # Calculate the Simple Moving Average (SMA)
    sma = data['Close'].rolling(window=window).mean()
    
    # Calculate the Standard Deviation
    std = data['Close'].rolling(window=window).std()
    
    # Calculate the Upper Bollinger Band
    upper_band = sma + (std * 2)
    
    # Calculate the Lower Bollinger Band
    lower_band = sma - (std * 2)
    
    # Print SMA, Upper Bollinger Band, and Lower Bollinger Band for visualization
    print("SMA:\n", sma.head())
    print("Upper Bollinger Band:\n", upper_band.head())
    print("Lower Bollinger Band:\n", lower_band.head())
    
    return sma, upper_band, lower_band

# Example usage:
# df = pd.DataFrame({'Close': [your_data_here]})
# sma, upper_band, lower_band = bb(df)


In [ ]:
def generate_signals(data, rsi_window=14, bollinger_window=20, macd_short=12, macd_long=26, macd_signal=9):
    """
    Generate buy and sell signals based on RSI, MACD, and Bollinger Bands indicators.
    
    Parameters:
    - data: pandas DataFrame with a 'Close' column and optionally 'RSI', 'MACD', 'Signal_Line', 'SMA', 'Upper_Band', 'Lower_Band'.
    - rsi_window: integer, the window period for RSI calculation.
    - bollinger_window: integer, the window period for Bollinger Bands calculation.
    - macd_short: integer, the short window period for MACD calculation.
    - macd_long: integer, the long window period for MACD calculation.
    - macd_signal: integer, the window period for Signal Line calculation.
    
    Returns:
    - data: pandas DataFrame with buy and sell signals appended.
    """
    # Calculate RSI
    if 'RSI' not in data:
        data['RSI'] = calc_RSI(data, window=rsi_window)
    
    # Calculate MACD and Signal Line
    if 'MACD' not in data or 'Signal_Line' not in data:
        macd, signal = calc_MACD(data, short_window=macd_short, long_window=macd_long, signal_window=macd_signal)
        data['MACD'] = macd
        data['Signal_Line'] = signal
    
    # Calculate Bollinger Bands
    if 'SMA' not in data or 'Upper_Band' not in data or 'Lower_Band' not in data:
        sma, upper_band, lower_band = bb(data, window=bollinger_window)
        data['SMA'] = sma
        data['Upper_Band'] = upper_band
        data['Lower_Band'] = lower_band
    
    # Generate buy signals
    data['Buy_Signal'] = (data['RSI'] < 30) & (data['MACD'] > data['Signal_Line']) & (data['Close'] < data['Lower_Band'])
    
    # Generate sell signals
    data['Sell_Signal'] = (data['RSI'] > 70) & (data['MACD'] < data['Signal_Line']) & (data['Close'] > data['Upper_Band'])
    
    # Print signals for visualization
    print("Signals:\n", data[['RSI', 'MACD', 'Signal_Line', 'SMA', 'Upper_Band', 'Lower_Band', 'Buy_Signal', 'Sell_Signal']].head())
    
    return data


In [ ]:
def run_backtest(data, initial_cash=10000):
    """
    Perform a backtest based on buy and sell signals.

    Parameters:
    - data: pandas DataFrame with 'Close' column and 'Buy_Signal', 'Sell_Signal', and 'Portfolio_Value' (optional) columns.
    - initial_cash: float, initial cash amount for the backtest.

    Returns:
    - data: pandas DataFrame with 'Portfolio_Value' column updated.
    - trades: list of tuples representing executed trades (action, timestamp, price).
    """
    cash = initial_cash
    shares = 1
    portfolio_values = []
    trades = []

    for i in range(len(data)):
        if data['Buy_Signal'].iloc[i]:
            if cash > 0:
                shares_to_buy = cash / data['Close'].iloc[i]
                cash -= shares_to_buy * data['Close'].iloc[i]
                shares += shares_to_buy
                trades.append(('Buy', data.index[i], data['Close'].iloc[i]))
        elif data['Sell_Signal'].iloc[i]:
            if shares > 0:
                cash += shares * data['Close'].iloc[i]
                shares = 0
                trades.append(('Sell', data.index[i], data['Close'].iloc[i]))
        portfolio_values.append(cash + shares * data['Close'].iloc[i])

    data['Portfolio_Value'] = portfolio_values
    return data, trades

In [ ]:


def calc_performance(data, trades):
    """
    Calculate returns and performance metrics for a backtested trading strategy.
    
    Parameters:
    - data: pandas DataFrame with 'Portfolio_Value' column.
    - trades: list of tuples representing executed trades (action, timestamp, price).
    
    Returns:
    - total_return: float, total return from the backtested strategy.
    - annual_return: float, annualized return from the backtested strategy.
    - sharpe_ratio: float, Sharpe ratio of the backtested strategy.
    - max_drawdown: float, maximum drawdown of the backtested strategy.
    """
    total_return = data['Portfolio_Value'].iloc[-1] - data['Portfolio_Value'].iloc[0]
    annual_return = (data['Portfolio_Value'].iloc[-1] / data['Portfolio_Value'].iloc[0]) ** (252 / len(data)) - 1
    daily_returns = data['Portfolio_Value'].pct_change().dropna()

    sharpe_ratio = np.nan
    if np.std(daily_returns) != 0:
        sharpe_ratio = np.mean(daily_returns) / np.std(daily_returns) * np.sqrt(252)

    max_drawdown = ((data['Portfolio_Value'].cummax() - data['Portfolio_Value']) / data['Portfolio_Value'].cummax()).max()
    
    print(f"Total Return: {total_return}, Annual Return: {annual_return}, Sharpe Ratio: {sharpe_ratio}, Max Drawdown: {max_drawdown}")
    
    return total_return, annual_return, sharpe_ratio, max_drawdown



In [54]:
def make_report(stock, total_return, annual_return, sharpe_ratio, max_drawdown):
    """
    Generate a performance report for a backtested trading strategy.
    
    Parameters:
    - stock: str, the name of the stock or asset.
    - total_return: float, total return from the backtested strategy.
    - annual_return: float, annualized return from the backtested strategy.
    - sharpe_ratio: float, Sharpe ratio of the backtested strategy.
    - max_drawdown: float, maximum drawdown of the backtested strategy.
    
    Returns:
    - report: dict, performance report containing the stock name and performance metrics.
    """
    report = {
        'Stock': stock,
        'Total_Return': total_return,
        'Annual_Return': annual_return,
        'Sharpe_Ratio': sharpe_ratio,
        'Max_Drawdown': max_drawdown
    }
    print(f"Report for {stock}:\n", report)
    return report

# Example usage:
# performance_report = make_report('AAPL', total_return, annual_return, sharpe_ratio, max_drawdown)


In [ ]:
def compile_reports(reports):
    """
    Compile a list of performance reports into a DataFrame and print a summary report.
    
    Parameters:
    - reports: list of dictionaries, each containing performance metrics for a stock.
    
    Returns:
    - summary: pandas DataFrame, summary report containing performance metrics for all stocks.
    """
    summary = pd.DataFrame(reports)
    print("Summary Report:\n", summary.head())
    return summary

# Example usage:
# summary_report = compile_reports([report1, report2, report3, ...])


In [ ]:


def plot_results(stock, data):
    """
    Visualize the results of backtesting and portfolio performance.
    
    Parameters:
    - stock: str, the name of the stock or asset.
    - data: pandas DataFrame with columns for 'Close', 'Upper_Band', 'Lower_Band', 'Buy_Signal', 'Sell_Signal', and 'Portfolio_Value'.
    """
    # Plot Close Price with Bollinger Bands and Buy/Sell Signals
    plt.figure(figsize=(14,7))
    plt.plot(data.index, data['Close'], label='Close Price')
    plt.plot(data.index, data['Upper_Band'], label='Upper Bollinger Band', linestyle='--')
    plt.plot(data.index, data['Lower_Band'], label='Lower Bollinger Band', linestyle='--')

    # Highlight Buy and Sell Signals
    buy_points = data.index[data['Buy_Signal'].fillna(False)]
    sell_points = data.index[data['Sell_Signal'].fillna(False)]
    plt.scatter(buy_points, data['Close'][data['Buy_Signal'].fillna(False)], label='Buy Signal', marker='^', color='green')
    plt.scatter(sell_points, data['Close'][data['Sell_Signal'].fillna(False)], label='Sell Signal', marker='v', color='red')

    plt.title(f'{stock} Price with Buy/Sell Signals')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    # Plot Portfolio Value Over Time
    plt.figure(figsize=(14,7))
    plt.plot(data.index, data['Portfolio_Value'], label='Portfolio Value', color='blue')
    plt.title(f'{stock} Portfolio Value Over Time')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value')
    plt.legend()
    plt.show()

# Example usage:
# plot_results('AAPL', df_backtest)


In [ ]:
def print_summary(reports):
    """
    Print a summary report based on performance reports of different stocks.
    
    Parameters:
    - reports: list of dictionaries, each containing performance metrics for a stock.
    """
    for report in reports:
        print(f"Stock: {report['Stock']}")
        print(f"Total Return: ${report['Total_Return']:.2f}")
        print(f"Annual Return: {report['Annual_Return']:.2%}")
        print(f"Sharpe Ratio: {report['Sharpe_Ratio']:.2f}")
        print(f"Max Drawdown: {report['Max_Drawdown']:.2%}")
        print("="*40)

# Example usage:
# print_summary([report1, report2, report3, ...])


In [ ]:
# Run the strategy
reports = []

In [ ]:
stock_data = get_stock_data(stocks, start_date, end_date)

In [ ]:
for stock, data in stock_data.items():
    # Generate Signals
    data = generate_signals(data)
    
    # Backtest
    data, trades = run_backtest(data)
    
    # Calculate Performance
    total_return, annual_return, sharpe_ratio, max_drawdown = calc_performance(data, trades)
    
    # Generate Report
    report = make_report(stock, total_return, annual_return, sharpe_ratio, max_drawdown)
    
    # Append to Reports
    reports.append(report)
    
    # Plot Results
    plot_results(stock, data)


In [ ]:
summary = compile_reports(reports)
print_summary(reports)

In [ ]:
reports_df = pd.DataFrame(reports)

# Save results to a CSV file
reports_df.to_csv('task1.csv', index=False)